# 15 minutes to QCoDeS

This short introduction is aimed mainly for beginners. Before you start with your first code using QCoDeS, make sure you have properly set up the Python environment for QCoDeS as explained in [this  document](http://qcodes.github.io/Qcodes/start/index.html#installation). 

## Introduction

QCodes is a python-based data acquisition and handling framework to facilitate experiments in nanoelectronics. As highly configurable open source project, we envision that this system may suite the needs of a diverse range of experimental setups, acting as a common system for regular experimental work across the community.

This guide offers a practical overview of Qcodes, going from installation to experimental data handling in a single notebook. Along the way links are provided to assist you in the configuration of this software's features for your experiments.

## Installation

QCoDeS is readily installed via pip or conda package managers in your preferred environment. These are other installation options are further detailed (in our installation guide)[https://qcodes.github.io/Qcodes/start/index.html#installation].

Install via pip:
> pip install qcodes

Install via conda:
> conda install qcodes

## Module imports

A wide range of modules are available for Qcodes, however for this example we will only import what is needed for a simple measurement.

In [ ]:
import numpy as np
import qcodes as qc
from qcodes import (
    Measurement,
    experiments,
    initialise_or_create_database_at,
    load_by_run_spec,
    load_or_create_experiment,
)
from qcodes.tests.instrument_mocks import (
    DummyInstrument, 
    DummyInstrumentWithMeasurement
)
from qcodes.utils.dataset.doNd import (
    dond, 
    plot, 
    LinSweep
)

from qcodes.dataset.plotting import plot_dataset


## Adding instruments

`Instrument` class in Qcodes is responsible for holding connections to hardware, creating a parameter or method for each piece of functionality of the instrument. For more information on instrument class we refer to the [detailed description here](http://qcodes.github.io/Qcodes/user/intro.html#instrument)  or the corresponding [api documentation](http://qcodes.github.io/Qcodes/api/instrument/index.html). 

Let us, now, create two dummy instruments and associate two parameters for each of them:

In [ ]:
# A dummy signal generator with two parameters ch1 and ch2
dac = DummyInstrument('dac', gates=['ch1', 'ch2'])

# A dummy digital multimeter that generates a synthetic data depending
# on the values set on the setter_instr, in this case the dummy dac
dmm = DummyInstrumentWithMeasurement('dmm', setter_instr=dac)

All instruments feature methods to enable you to inspect their configuration. We refer to this as a ``snapshot``. For convenience, methods are provided for a human readable version allowing us to take a glance at our digital multimeter:

In [ ]:
dmm.print_readable_snapshot()

As we can see here, our dummy multimeter, `dmm`, has two `parameters` (v1 and v2), that correspond the two channels of our dummy signal generator `dac`. 

## Parameters

A QCoDeS `Parameter` is a value from an instrument that may get and/or set values by methods. Intuitively this is how Qcodes communicates with most instrumentation, for example a digital multimeter contains settings (e.g. mode, range) and provide data (e.g. voltage, current). These methods are defined within the instrument drivers that are provided by the community, [with many examples available](https://qcodes.github.io/Qcodes/examples/index.html#drivers).

In our current example we are using dummy instruments with trivial `set()` and `get()` methods to generate synthetic data. For the dac, these settable `parameters` are added in the instantiation of the `DummyInstrument` class (i.e. `ch1` and `ch2`). Similarly, the dummy digtal multimeter, dmm, has gettable `parameters` added by the instantiation of the `DummyInstrumentWithMeasurement` class defined by the output channels of the setter instrument (i.e. the dac). 


For convenience QCoDeS provides following parameter classes built in: 

   - `Parameter` : Represents a single value at a given time. Example: voltage. 
   - `ParameterWithSetpoints`: Represents an array of values of all the same type that are returned all at once. Example: voltage vs time waveform . We refer to the [notebook](http://qcodes.github.io/Qcodes/examples/Parameters/Simple-Example-of-ParameterWithSetpoints.html) in which more detailed examples concerning the use cases of this parameter can be found.
   - `DelegateParameter`: It is intended for proxy-ing other parameters. You can use different label, unit, etc in the delegated parameter as compared to the source parameter.
   - `MultiParameter`: Represents a collection of values with different meanings and possibly different dimensions. Example: I and Q, or I vs time and Q vs time.

These built in parameter classes are typically used as a wrapper for instrument communications. The user-facing `set()` and `get()` methods calling instrument facing `set_raw()` and `get_raw()` commands. Futher exammples of these parameters are discussed in our example [notebook on Parameters](http://qcodes.github.io/Qcodes/examples/index.html#parameters). 

### Examples
In most cases, a settable parameter accepts its value as a function argument. 

Let us set the a value of 1.1 for the `ch1` parameter of our signal generator, `dac`, by this approach:

In [ ]:
dac.ch1(1.1)

Similarly, a gettable parameter will often return its value with a simple function call.

For example, we can read the value of our digital multimeter, `dmm`, like so:

In [ ]:
dmm.v1()

## Stations  

A station is a collection of all the instruments and devices present in your experiment. As mentioned earlier, it can be thought of as a bucket where you can add your `instruments`, `parameters` and other `components`. Each of these terms has a definite meaning in QCoDeS and shall be explained in later sections. Once a station is properly configured, you can use its instances to access these components. We refer to tutorial on [Station](http://qcodes.github.io/Qcodes/examples/Station.html) for more details.

To organize our dummy instruments, we will first instantiate a station as so:

In [ ]:
station = qc.Station()

### Adding instruments to the station

Every instrument that you are working with during an experiment should be added to a `Station`.  

Here, we add the `dac` and `dmm` instruments by using our station's `add_component()` method: 

In [ ]:
station.add_component(dac)
station.add_component(dmm)

### Inspecting the station

For any experiment it is essential to have a record of the instrumental setup. To enable this, a `station` has a `snapshot()` method which provides a dictionary of all `instruments` and the properties associated with them (e.g. `parameters`) in a recursive manner.


In [ ]:
station.snapshot()

### Saving and loading configurations.

The instantiation of the instruments, that is, setting up the proper initial values of the corresponding parameters and similar pre-specifications of a measurement constitutes the initialization portion of the code. In general, this portion can be quite long and tedious to maintain. These (and more) concerns can be solved by a YAML configuration file of the `Station` object. We refer to the notebook on [station](http://qcodes.github.io/Qcodes/examples/Station.html#Default-Station) for more details.

(note to add an example YAML file of the above configuration)

## Databases and experiments.

With `station` a working station, the next step is to set up a `database` in order to save our data to. In QCodes, we implement a SQLite3 database for this purpose. 

### Initialize or create a database

Before starting a measurement, we first initialise a database. The location of the database is specified by the configuration object of the QCoDeS installation. The database is created with the latest supported version complying with the QCoDeS version that is currently under use. If a database already exists but an upgrade has been done to the QCoDeS, then that database can continue to be used and it is going to be upgraded to the latest version automatically at first connection.

The initialisation (or creation) of the database at a particular location is achieved via static function:

In [ ]:
initialise_or_create_database_at("~/experiments.db")

By default, QCodes only supports a single active database. The current database location is stored in the QCodes `configuration`. 

In [ ]:
qc.config.core.db_location

### Load or create an experiment

After initialising the database we create an `Experiment` object. This object contains the names of the experiment and sample, and acts as a manager for data acquired during `Measurements`. The `load_or_create_experiment` function will return an existing experiment with the same name, but if no experiments are found, it will create a new one.

For this example, we will call our experiment `exp`:


In [ ]:
tutorial_exp = load_or_create_experiment(experiment_name='15_minute_example',
                                sample_name="synthetic data")

The path of the database for `Experiment` is the defined path in the QCoDeS configuration. First, `Experiment` loads the database in that path (or it creates one if there is no database in that path), and then saves the created experiment in that database. Although loading/ creating database by `Experiment` is a user-friendly feature, we recommend users to initialise their database, as shown earlier, before loading/ creating their experiment, because it allows them to better control their experiments and databases for their measurement.

The method shown above to load or create the experiment is the most versatile one. However there are other options discussed in the guide on databases.

## Measurements

The `measurement` object is used to obtain data from instruments in QCodes, as such it is instantiated with both an `experiment` (to handle data) and `station` to control the instruments. If these arguments are absent, the most recent experiment and station are used as defaults. A keyword argument `name` can also be set as any string value, this string will be used to identify the resulting dataset. 

In [ ]:
context_meas = Measurement(exp=tutorial_exp, station=station, name='context_example')

It is possible to instantiate a `measurement` prior to creating or loading an experiment, but this is not advisable.
1. If the initialised `database` does not contain an `experiment`, then the instantiaion will raise an error and halt your work.  
2. If the database already contains an `experiment`, then the instantiated `measurement`  will be added to the most recent `experiment` in the database without raising an error message or warning. This will lead to poor data management.


### Registering parameters to measure

Qcodes features the ability to store the relationship between parameters (i.e. parameter `y` is dependent on `x`). This feature allows the intent of the measurement to be clearly recorded in the expriemental records. In addition, the parameter dependency is used to define the coordinate axes when plotting the data using qcodes. The parameters which are being measured are first registered with the `measurement`. When registering a dependent parameter (i.e. y(x)) the independent parameter is declared as a setpoint. As a consequence, independent parameters must be registered prior to their corresponding dependent parameters. 

In our example, ``dac.ch1`` is the independent parameter and ``dmm.v1`` is the dependent parameter. So we register ``dmm.v1`` with the setpoint as ``dac.ch1``. 

In [ ]:
context_meas.register_parameter(dac.ch1)                       # Register the independent parameter...
context_meas.register_parameter(dmm.v1, setpoints=(dac.ch1,))  # ...then register the dependent parameter

### Measuring with the measurement context manager

Qcodes `Measurement` module provides a context manager for registering parameters to measure and store results. Within the context manager, measured data is periodically saved to the `database` as a background process.

To conduct a simple measurement, we can create a simple loop inside the context manager which will control the instruments, acquire data, and store the results. This is the most configurable way of acquiring data using Qcodes.


In [ ]:
context_meas.write_period = 2                                  # Time for periodic background database writes

with context_meas.run() as datasaver:
    for set_v in np.linspace(0, 25, 10):
        dac.ch1.set(set_v)
        get_v = dmm.v1.get()
        datasaver.add_result((dac.ch1, set_v),
                             (dmm.v1, get_v))

    dataset = datasaver.dataset                        # Convenient to have for plotting

The ``meas.run()`` method returns a context manager to control data acquisition and storage. Entering the context provides a ``DataSaver`` object, which we will store in the ``datasaver`` variable. Using a simple loop structure, we can use instruments' ``set`` and ``get`` methods to control the instrument and acquire data respectively. The ``add_result`` method validates the sizes of all the data points and store them intermittently into a cache. Within every write-period of the measurement, the data of this cache is flushed to the database in a background thread.

### Measuring multidimensional data with doNd

Qcodes also includes functions to produce multidimensional data sets with [optimized data handling](https://qcodes.github.io/Qcodes/examples/DataSet/Using_doNd_functions_in_comparison_to_Measurement_context_manager_for_performing_measurements.html); of these, ``dond`` (i.e. do n-dimensional) is facilitates collecting multidimensional data.  Similar optimizations can be made using the measurement context (see [measuring with shaped data](https://qcodes.github.io/Qcodes/examples/DataSet/Performing-measurements-using-qcodes-parameters-and-dataset.html#Specifying-shape-of-measurement)), but this approach simplifies the setup and readability of the code. 

We will first set up the measurement by defining the sweeps for each independent parameters, in our case the two channels of ``dac``:

In [ ]:
# Setting up a doNd measurement
sweep_1 = LinSweep(dac.ch1, -1, 1, 20, 0.01)
sweep_2 = LinSweep(dac.ch2, -1, 1, 20, 0.01)


This linear sweeps for `dac.ch1` and `dac.ch2` are defined by the endpoints of the sweep (-1 to 1 V), the number of steps (20) and a time delay between each step (0.01 s). This delay time is used to allow real instruments to equilibriate between each step in the sweep. Multiple types of sweeps are [included with Qcodes](https://qcodes.github.io/Qcodes/examples/DataSet/Using_doNd_functions_in_comparison_to_Measurement_context_manager_for_performing_measurements.html) to enable a variety of sampling schemes. 

In this case, we do not need to register parameters, this is done automatically by the dond function:

In [ ]:
dond(
    sweep_1,                            # 1st independent parameter
    sweep_2,                            # 2nd independent parameter
    dmm.v1,                             # 1st dependent parameter
    dmm.v2,                             # 2nd dependent parameter
    measurement_name="dond_example",     # Set the measurement name
    exp=tutorial_exp,                   # Set the experiment to save data to.
    show_progress=True                  # Optional progress bar
    )


The ``dond`` function features a number of options (e.g. plotting, database write period, multithreading) which are further detailed. In short, complex multidimensional data may be acquired without deep nested loop structures.

## Data exploration, plotting, and export

In this section we detail methods and functions for working with data.

### List all datasets in a database.

The list of experiments that are stored in the database can be called back as follows:

In [ ]:
experiments()

While this example database contains only a few experiments this number may grow significantly as you perform measurements on your nanoelectronic devices. 

While our example database contains only few experiments, in reality the database will contain several experiments containing many datasets. Often, you would like to load a dataset from a particular experiment for further analysis. Here we shall explore different ways to find and retrieve already measured dataset from the database.

### List all the datasets in an experiment

Let us now retrieve the datasets stored within the current experiment via:

In [ ]:
tutorial_exp.data_sets()

### Load the data set using one or more specifications

The method ``load_by_run_spec`` can be used to load a run with given specifications such as 'experiment name' and 'sample name':

In [ ]:
dataset = load_by_run_spec(experiment_name='15_minute_example', captured_run_id=88)

While the arguments are optional, the function call will raise an error if more than one run matching the supplied specifications is found. If such an error occurs, the traceback will contain the specifications of the runs, as well. Further information concerning 'Uniquely identifying and loading runs' can be found in [this example notebook](DataSet/Extracting-runs-from-one-DB-file-to-another.ipynb#Uniquely-identifying-and-loading-runs).

For more information on the `DataSet` object that `load_by_run_spec` returned, refer to [DataSet class walkthrough article](DataSet/DataSet-class-walkthrough.ipynb).

### Plot dataset 

We arrived at a point where we can visualize our data. To this end, we use the ``plot_dataset`` method with ``dataset`` as its argument:

In [ ]:
plot_dataset(dataset)

For more detailed examples of plotting QCoDeS datasets, refer to the following articles:

- [Offline plotting tutorial](DataSet/Offline%20Plotting%20Tutorial.ipynb)
- [Offline plotting with categorical data](DataSet/Offline%20plotting%20with%20categorical%20data.ipynb)
- [Offline plotting with complex data](DataSet/Offline%20plotting%20with%20complex%20data.ipynb)

### Get data of specific parameter of a dataset

If you are interested in numerical values of a particular parameter within a given dataset, the corresponding data can be retrieved by using `get_parameter_data` method:

In [ ]:
dataset.get_parameter_data('dac_ch1')

In [ ]:
dataset.get_parameter_data('dmm_v1')

We refer reader to [exporting data section of the performing measurements using qcodes parameters and dataset](DataSet/Performing-measurements-using-qcodes-parameters-and-dataset.ipynb#Accessing-and-exporting-the-measured-data) and [Accessing data in DataSet notebook](DataSet/Accessing-data-in-DataSet.ipynb) for further information on `get_parameter_data` method.

### Export data to pandas dataframe

If desired, any data stored within a QCoDeS database can also be exported as pandas dataframes. This can be achieved via:

In [ ]:
df = dataset.to_pandas_dataframe_dict()['dmm_v1']
df.head()

### Export data to xarray

It's also possible to export data stored within a QCoDeS database to an `xarray.DataArray`. This can be achieved via:

In [ ]:
xarray = dataset.to_xarray_dataarray_dict()['dmm_v1']
xarray.head()

We refer to [example notebook on working with pandas](DataSet/Working-With-Pandas-and-XArray.ipynb) and [Accessing data in DataSet notebook](DataSet/Accessing-data-in-DataSet.ipynb) for further information.

### Explore the data using an interactive widget

Experiments widget presents the most important information at a glance, has buttons to plot the dataset and easily explore a snapshot, enabled users to add a note to a dataset.

It is only available in the Jupyter notebook because it uses [`ipywidgets`](https://ipywidgets.readthedocs.io/) to display an interactive elements.

Use it in the following ways:
```python
# import it first
from qcodes.interactive_widget import experiments_widget

# and then just run it
experiments_widget()  

# you can pass a specific database path
experiments_widget(db="path_of_db.db")

# you can also pass a specific list of DataSets:
# say, you're only interested in datasets of a particular experiment
experiments = qcodes.experiments()
data_sets = experiments[2].data_sets()
experiments_widget(data_sets=data_sets)

# you can change the sorting of the datasets
# by passing None, "run_id", "timestamp" as sort_by argument:
experiments_widget(sort_by="timestamp")
```

Here's a short video that summarizes the looks and the features:

![video demo about experiments widget should show here](../_static/experiments_widget.webp)

## Things to remember

### QCoDeS configuration  

QCoDeS uses a JSON based configuration system. It is shipped with a default configuration. The default config file should not be overwritten. If you have any modifications, you should save the updated config file on your home directory or in the current working directory of your script/notebook. The QCoDeS config system first looks in the current directory for a config file and then in the home directory for one and only then - if no config files are found - it falls back to using the default one. The default config is located in `qcodes.config`. To know how to change and save the config please refer to the [documentation on config](http://qcodes.github.io/Qcodes/user/configuration.html?).

### QCoDeS instrument drivers

We support and provide drivers for most of the instruments currently in use at the Microsoft stations. However, if more functionalities than the ones which are currently supported by drivers are required, one may update the driver or request the features form QCoDeS team. You are more than welcome to contribute and if you would like to have a quick overview on how to write instrument drivers, please refer to the [example notebooks on writing drivers](http://qcodes.github.io/Qcodes/examples/index.html#writing-drivers).

### QCoDeS measurements live plotting with Plottr

Plottr supports and is recommended for QCoDeS measurements live plotting. [How to use plottr with QCoDeS for live plotting](plotting/How-to-use-Plottr-with-QCoDeS-for-live-plotting.ipynb) notebook contains more information.